In [152]:
# !pip install numpy
# !pip install pandas
# !pip install plotly
# !pip install nbformat --upgrade
# !pip install yfinance
# !pip install dash
# !pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:02


In [1]:
import yfinance as yf
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import kaleido

pio.renderers.default = 'browser' # plotly renders in browser

/var/folders/0c/6nxq1mwn3g99r6b2tcmbw3xw0000gn/T/ipykernel_86466/3338090756.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/0c/6nxq1mwn3g99r6b2tcmbw3xw0000gn/T/ipykernel_86466/3338090756.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
# GLOBAL DATA CONFIGS
tickers = ['AAPL', 'GOOG', 'MSFT', 'TSLA', 'NVDA', 'BTC-USD'] 
start_date = '2020-01-01'
end_date = '2024-05-05'

In [3]:
def fetch_stock_data(tickers, start_date=None, end_date=None, interval='1d'):
    """
    Fetches historical stock data for given tickers using yfinance.
    
    Parameters:
    - tickers (list of str): List of stock ticker symbols.
    - start_date (str): Start date for the data in format 'YYYY-MM-DD' (optional).
    - end_date (str): End date for the data in format 'YYYY-MM-DD' (optional).
    - interval (str): Data interval. Valid intervals: '1d', '1wk', '1mo', etc.

    Returns:
    - dict: A dictionary with tickers as keys and DataFrames as values.
    """
    stock_data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date, interval=interval)
        stock_data[ticker] = data
    
    return stock_data

data = fetch_stock_data(tickers, start_date, end_date, '1d')

In [4]:
data['NVDA'].head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-02 00:00:00-05:00,59.452387,59.741245,58.946886,59.741245,23753600,0.0,0.0
2020-01-03 00:00:00-05:00,58.543481,59.223292,58.294466,58.785027,20538400,0.0,0.0
2020-01-06 00:00:00-05:00,57.851210,59.083835,57.589744,59.031540,26263600,0.0,0.0
2020-01-07 00:00:00-05:00,59.315428,60.204414,58.864711,59.746223,31485600,0.0,0.0
2020-01-08 00:00:00-05:00,59.703892,60.271646,59.302977,59.858284,27710800,0.0,0.0


In [5]:
# Compute additional metrics
def compute_metrics(data):
    """
    Computes additional metrics for stock data.
    
    Parameters:
    - data (dict): A dictionary with tickers as keys and DataFrames as values.
    
    Returns:
    - dict: A dictionary with tickers as keys and DataFrames as values.
    """
    metrics = {}
    for ticker, df in data.items():
        # Calculate daily return
        df['Daily Return'] = df['Close'].pct_change().fillna(0)
        
        # Calculate cumulative return
        df['Cumulative Return'] = (1 + df['Daily Return']).cumprod().fillna(1)
        
        # Calculate simple moving averages for specified periods
        df['SMA_3'] = df['Close'].rolling(window=3).mean()
        df['SMA_7'] = df['Close'].rolling(window=7).mean()
        df['SMA_30'] = df['Close'].rolling(window=30).mean()  # Approximating 1 month
        df['SMA_90'] = df['Close'].rolling(window=90).mean()  # Approximating 3 months
        
        # Calculate VWAP (Volume Weighted Average Price)
        if 'Volume' in df.columns:
            df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
            df['VWAP_3'] = (df['Close'] * df['Volume']).rolling(window=3).sum() / df['Volume'].rolling(window=3).sum()
            df['VWAP_7'] = (df['Close'] * df['Volume']).rolling(window=7).sum() / df['Volume'].rolling(window=7).sum()
            df['VWAP_30'] = (df['Close'] * df['Volume']).rolling(window=30).sum() / df['Volume'].rolling(window=30).sum()
            df['VWAP_90'] = (df['Close'] * df['Volume']).rolling(window=90).sum() / df['Volume'].rolling(window=90).sum()
        
        metrics[ticker] = df
    
    return metrics

In [6]:
data = compute_metrics(data)
data['NVDA'].head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily Return,Cumulative Return,SMA_3,SMA_7,SMA_30,SMA_90,VWAP,VWAP_3,VWAP_7,VWAP_30,VWAP_90
Date,,,,,,,,,,,,,,,,,,
2020-01-02 00:00:00-05:00,59.452387,59.741245,58.946886,59.741245,23753600,0.0,0.0,0.000000,1.000000,NaN,NaN,NaN,NaN,59.741245,NaN,NaN,NaN,NaN
2020-01-03 00:00:00-05:00,58.543481,59.223292,58.294466,58.785027,20538400,0.0,0.0,-0.016006,0.983994,NaN,NaN,NaN,NaN,59.297842,NaN,NaN,NaN,NaN
2020-01-06 00:00:00-05:00,57.851210,59.083835,57.589744,59.031540,26263600,0.0,0.0,0.004193,0.988120,59.185937,NaN,NaN,NaN,59.198714,59.198714,NaN,NaN,NaN
2020-01-07 00:00:00-05:00,59.315428,60.204414,58.864711,59.746223,31485600,0.0,0.0,0.012107,1.000083,59.187597,NaN,NaN,NaN,59.367652,59.254299,NaN,NaN,NaN
2020-01-08 00:00:00-05:00,59.703892,60.271646,59.302977,59.858284,27710800,0.0,0.0,0.001876,1.001959,59.545349,NaN,NaN,NaN,59.472435,59.562923,NaN,NaN,NaN


# Comparative Stock Visualizer

Description:

    This function visualizes the calculated statistics for multiple stocks over a specified date range using Plotly, an interactive graphing library. It creates a line chart where each line represents the normalized growth trajectory of a different stock. This allows for easy comparison of stock performance over time.

Output:

    An interactive line chart will be displayed. The chart is rendered in a web browser for an enhanced viewing experience, providing tools for zooming, panning, and toggling data series visibility.



In [7]:
def plotly_cumulative_growth(dataframes, start_date, end_date, initial_metric='Cumulative Return'):
    fig = go.Figure()

    # Convert user input dates to timezone-naive datetime
    start_date_naive = pd.to_datetime(start_date).tz_localize(None)
    end_date_naive = pd.to_datetime(end_date).tz_localize(None)

    metrics = ['Cumulative Return', 'Close', 'Volume', 'Daily Return', 'SMA_3', 'SMA_7', 'SMA_30', 'SMA_90', 'VWAP', 'VWAP_3', 'VWAP_7', 'VWAP_30', 'VWAP_90']

    # Initialize the list for all traces
    all_traces = []

    for stock, df in dataframes.items():
        # Ensure the Date column is correct
        if 'Date' not in df.columns:
            df = df.reset_index()

        df['Date'] = pd.to_datetime(df['Date']).dt.tz_localize(None)

        # Filter the data frame based on the input date range
        mask = (df['Date'] >= start_date_naive) & (df['Date'] <= end_date_naive)
        filtered_df = df.loc[mask]

        # Generate traces for each metric
        for metric in metrics:
            trace = go.Scatter(
                x=filtered_df['Date'],
                y=filtered_df[metric],
                mode='lines',
                name=f"{stock} {metric}",
                visible=(metric == initial_metric)  # Only make the initial metric visible
            )
            all_traces.append(trace)
        
    # Add all traces to the figure
    for trace in all_traces:
        fig.add_trace(trace)

    # Create the button list for the dropdown
    buttons = []
    for metric in metrics:
        # Create a list for visibility, setting only the corresponding metric traces to True
        visibility = [metric in trace.name for trace in all_traces]
        buttons.append(
            dict(
                label=metric,
                method='update',
                args=[{'visible': visibility},
                      {'title': f'{metric} of Stocks Over Time',
                       'yaxis': {'title': metric}}]
            )
        )
    
    for ticker in dataframes.keys():
        # Add a button for each stock to toggle visibility
        visibility = [ticker in trace.name for trace in all_traces]
        buttons.append(
            dict(
                label=ticker,
                method='update',
                args=[{'visible': visibility},
                      {'title': f'{initial_metric} of Stocks Over Time',
                       'yaxis': {'title': initial_metric}}]
            )
        )

    # Add the dropdown menu to the layout
    fig.update_layout(
        title=f'{initial_metric} of Stocks Over Time',
        xaxis_title='Date',
        yaxis_title=initial_metric,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(step="all")
                ]),
                bgcolor='#333',  # Background color
                activecolor='#666',  # Active button background color
                bordercolor='#666',  # Border color
                font=dict(color='white')  # Text color
            ),
            rangeslider=dict(visible=True),
            type="date"
        ),
        updatemenus=[{
            'buttons': buttons,
            'direction': 'down',
            'pad': {'r': 10, 't': 10},
            'showactive': True,
            'x': 0.5,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }],
        template="plotly_dark"
    )

    # Display the figure
    fig.show()


plotly_cumulative_growth(data, "2020-01-02", "2024-05-05", 'Cumulative Return')

In [166]:
def plot_violin(dataframes):
    fig = go.Figure()

    for stock, df in dataframes.items():
        # Assuming 'Daily Return' is already calculated and exists in the DataFrame
        fig.add_trace(go.Violin(y=df['Daily Return'].dropna(),  # Drop NA values for clean plotting
                                name=stock,  # Use the stock ticker as the name
                                box_visible=True,  # Show the inner box plot
                                meanline_visible=True))  # Show the mean line within the violin

    # Update plot layout
    fig.update_layout(
        title='Distribution of Daily Returns for Each Stock',
        yaxis_title='Daily Returns',
        xaxis=dict(
            title='Stock Tickers',
            type='category'
        ),
        template="plotly_dark"
    )
    # display the figure in the code editor
    fig.show(renderer='browser')

# Example usage:
plot_violin(data)